<a href="https://colab.research.google.com/github/SreeKumara/Generative-AI-2025/blob/main/GenAi_Lab_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import zipfile
from PIL import Image
import io
import os

train_data_path = "/content/train-20250404T145129Z-001.zip"
validation_data_path = "/content/validation-20250404T145130Z-001.zip"

# Image preprocessing and data augmentation
image_size = (150, 150)  # Adjust as needed
batch_size = 32

# Function to load images from zip file
def load_images_from_zip(zip_path, target_size):
    images = []
    labels = []
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            if file_info.filename.endswith(('.png', '.jpg', '.jpeg')):  # Adjust file extensions as needed
                # Get class label from folder name
                class_label = os.path.basename(os.path.dirname(file_info.filename))

                # Read image data and resize
                with zip_ref.open(file_info.filename) as image_file:
                    image = Image.open(io.BytesIO(image_file.read()))
                    image = image.resize(target_size)
                    image = np.array(image)
                    images.append(image)
                    labels.append(class_label)

    return np.array(images), np.array(labels)

# Load training and validation data
train_images, train_labels = load_images_from_zip(train_data_path, image_size)
validation_images, validation_labels = load_images_from_zip(validation_data_path, image_size)

# Preprocess images (rescale pixel values)
train_images = train_images / 255.0
validation_images = validation_images / 255.0

# One-hot encode labels (if needed)
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
validation_labels_encoded = label_encoder.transform(validation_labels)

num_classes = len(label_encoder.classes_)
train_labels_onehot = to_categorical(train_labels_encoded, num_classes=num_classes)
validation_labels_onehot = to_categorical(validation_labels_encoded, num_classes=num_classes)

# CNN Model Architecture
def create_model():
  model = keras.Sequential([
      layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(128, (3, 3), activation='relu'),
      layers.MaxPooling2D((2, 2)),
      layers.Conv2D(256, (3,3), activation='relu'),
      layers.MaxPooling2D((2,2)),
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(num_classes, activation='softmax')
  ])
  return model

# Create and compile the model
model = create_model()
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
history = model.fit(
    train_images,
    train_labels_onehot,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(validation_images, validation_labels_onehot)
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_images, validation_labels_onehot, verbose=0)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5000 - loss: 0.6920 - val_accuracy: 0.5000 - val_loss: 0.6927
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7500 - loss: 0.6891 - val_accuracy: 0.5000 - val_loss: 0.6929
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.6861 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.6833 - val_accuracy: 0.5000 - val_loss: 0.6936
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.6807 - val_accuracy: 0.5000 - val_loss: 0.6941
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7500 - loss: 0.6780 - val_accuracy: 0.5000 - val_loss: 0.6947
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.7500 - loss: 0.6754 - val_accuracy: 0.5000 - val_loss: 0.6953
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7500 - loss: 0.6730 - val_accuracy: 0.5000 - val_loss: 0.6959
Epoch 9/10
1/1 ━

In [5]:
# 1. Upload files to Colab manually
from google.colab import files

uploaded = files.upload()  # Upload the two zip files here


Saving train-20250404T145129Z-001.zip to train-20250404T145129Z-001 (1).zip
Saving validation-20250404T145130Z-001.zip to validation-20250404T145130Z-001 (1).zip


In [6]:
import os
import zipfile

# 2. Extract zip files to local directories
def unzip_dataset(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

unzip_dataset('train-20250404T145129Z-001.zip', 'dataset/train')
unzip_dataset('validation-20250404T145130Z-001.zip', 'dataset/val')

# 3. Paths
train_dir = 'dataset/train'
val_dir = 'dataset/val'

In [7]:
# ✅ STEP 3: Preprocessing
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_HEIGHT, IMG_WIDTH = 150, 150
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1.0/255)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 4 images belonging to 1 classes.
Found 4 images belonging to 1 classes.


In [8]:
# ✅ STEP 4: Build CNN (based on Tabela 1)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def build_model():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(256, activation='relu'),
        Dense(train_generator.num_classes, activation='softmax')
    ])
    return model


In [9]:
# ✅ STEP 5: Compile and Train
model = build_model()
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

EPOCHS = 10

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.f

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_acc

In [10]:
# ✅ STEP 6: Evaluate
train_loss, train_acc = model.evaluate(train_generator)
val_loss, val_acc = model.evaluate(val_generator)

print(f"\n✅ Training Accuracy: {train_acc:.4f}")
print(f"✅ Validation Accuracy: {val_acc:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - accuracy: 1.0000 - loss: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 1.0000 - loss: 0.0000e+00

✅ Training Accuracy: 1.0000
✅ Validation Accuracy: 1.0000
